In [ ]:
!pip install -U classiq

In [39]:
import classiq

In [3]:
from classiq import *
import numpy as np
from classiq import synthesize
from classiq import show
import math

In [26]:
from classiq import (
    CInt,
    Output,
    QArray,
    QBit,
    X,
    allocate,
    control,
    create_model,
    hadamard_transform,
    invert,
    power,
    qft,
    qfunc,
    repeat,
    unitary,
)

In [30]:
from classiq import (
    CCX,
    CPHASE,
    CX,
    PHASE,
    CInt,
    H,
    Output,
    Preferences,
    QArray,
    QBit,
    X,
    allocate,
    apply_to_all,
    bind,
    control,
    create_model,
    invert,
    qft,
    qfunc,
    repeat,
    show,
    synthesize,
)

from classiq.qmod.symbolic import pi

@qfunc
def my_qft_step(qbv: QArray[QBit]) -> None:
    H(qbv[0])
    repeat(
        count=qbv.len - 1,
        iteration=lambda index: CPHASE(pi / 2 ** (index + 1), qbv[0], qbv[index + 1]),
    )


# qft without SWAP gates
@qfunc
def qft_ns(qbv: QArray[QBit]) -> None:
    repeat(
        count=qbv.len,
        iteration=lambda index: my_qft_step(qbv[index : qbv.len]),
    )

@qfunc
def apply_rotations_diff_ijl(N: CInt, i: CInt, j:CInt, ctrl1: QBit, ctrl2:QBit , y:QArray):
  ctrl = QArray('ctrl')
  bind([ctrl1, ctrl2] , ctrl)
  # Calculate the phase angle separately and then pass it to PHASE
  control(ctrl, lambda: repeat(Y_S , lambda k : PHASE((2* pi *(2**(i+j+k))/ N) % (2 * pi), y[k])))
  bind(ctrl, [ctrl1, ctrl2])

@qfunc
def apply_rotations1(N: CInt , i: CInt, ctrl1: QBit, y:QArray)  :
          control(ctrl1, lambda: repeat(Y_S, lambda k: PHASE(2* pi * ((2**(2*i+k))%N) /N,y[k])))

@qfunc
def modular_square(N: CInt, x: QArray[QNum], y: QArray[QNum]):
  hadamard_transform(y)
  for i in range(X_S):
    apply_rotations1(N, i, x[i], y)
    for j in range(i+1, X_S):
      apply_rotations_diff_ijl(N,i,i+j+1, x[i],x[j], y)
    invert(lambda: qft(y))


In [ ]:
N=8
Y_S=int(np.ceil(np.log2(N)))
X_S=Y_S
print(N, Y)

In [47]:
@qfunc
def main(x: Output[QArray], y: Output[QArray]):
  N=8
  allocate(X_S , x)
  allocate(Y_S,y)
  within_apply(within=lambda: qft(x), apply = lambda: qft(x)) # Added 'within' argument
  modular_square(N, x, y)
qmod = create_model(main)

In [ ]:
from classiq import show, synthesize

qprog = synthesize(qmod)
show(qprog)

In [ ]:
from classiq import execute

results = execute(qprog).result()
res = results[0].value

print(res.parsed_states)
